# Clustering "Toronto" Neighbourhoods into 4 distincs clusters (Central, Down-Town, East & West Toronto) and looking at the venues in Central Toronoto using Foursqaure APIs

Based on the instruction on the course assignment page as well as on this website: https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722.

In [58]:
import requests # Need to get wikipedia contents
import lxml.html as lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  # import Beautifulsoup # Get BeautifulSoup package
print('BeautifulSoup imported')

BeautifulSoup imported


In [41]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  # URL of the target wikipedia page

In [42]:
# Using BeautifulSoup to scrape the wikipedia page
website_URL=requests.get(url).text
#website_URL

In [43]:
#Create 
soup = BeautifulSoup(website_URL,"lxml")

In [44]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [47]:
Table = soup.find("table",{"class":"wikitable sortable"}) 

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

In [48]:
doc

<Element html at 0x1afff94f318>

In [51]:
withlink=Table.findAll('a')

In [53]:
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]
tr_elements = doc.xpath('//tr')

In [55]:
tr_elements[1:5]

[<Element tr at 0x1afffb92138>,
 <Element tr at 0x1afffb92318>,
 <Element tr at 0x1afffb92278>,
 <Element tr at 0x1afffb92368>]

In [54]:
#Create empty list
Dict=[]
col=[]
i=0

#For the first row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [56]:
#first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1    
[len(C) for (title,C) in col]
Dict={title:column for (title,column) in col}   # Dictionary with Postcodes, Borough and Neighbourhoods
df=pd.DataFrame(Dict)  # Creating the pandas Dataframe using the dictionary values

In [61]:
df.head()
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

In [62]:
df_Borough=df[df.Borough != 'Not assigned']   # get rid of "Not assigned" boroughs

In [63]:
df_Borough.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [81]:
df_Borough_new=df_Borough[['Postcode', 'Borough']].copy()
df_Borough_new.loc[:,'Neighbourhood'] =pd.Series(df_Borough['Neighbourhood'])# pd.Series(np.random.randn(sLength), index=df1.index)
#df_Borough_new['Neighbourhood']=s.str.slice(stop=-1)

In [82]:
df_Borough_new.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwood
3,M4A,North York,Victoria Villag
4,M5A,Downtown Toronto,Harbourfron
5,M5A,Downtown Toronto,Regent Par
6,M6A,North York,Lawrence Height


In [91]:
df_Borough_new.loc[df_Borough_new.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_Borough_new.Borough

In [84]:
df_new=df_Borough_new.groupby('Postcode', sort=False)['Neighbourhood'].apply(','.join)  # group neighbourhoods by Postcode

In [86]:
df_new_FINAL=pd.DataFrame({'Postcode':df_new.index, 'df_new':df_new.values})
df_new_F=df_new_FINAL.rename(columns={ df_new_FINAL.columns[1]: "Neighbourhood" })
df_N=df_new_F['Neighbourhood'].tolist() # create a neighbourhood list

In [87]:
df_Borough_BONLY=df_Borough['Borough'] # create a Borough lit
df_Borough_PONLY=df_Borough['Postcode'] # create a postcode list

In [88]:
df_Borough_BandP=pd.concat([df_Borough_PONLY, df_Borough_BONLY], axis=1)
df_Borough_BandP_new = df_Borough_BandP.drop_duplicates(subset=['Postcode','Borough'])  # drop duplicates

In [89]:
df_P=df_Borough_BandP_new['Postcode'].tolist() # refined lists
df_B=df_Borough_BandP_new['Borough'].tolist() # refined lists

In [90]:
Toronto_DICT= {'Postcode':df_P,'Borough':df_B, 'Neighbourhood':df_N} # Create a dictionary with the 3 refined lists
Toronto_df=pd.DataFrame(Toronto_DICT) # create the dataframe
# "List of Postcodes, Boroughs, & Neighbourhoods in TORONTO FOR 103 POSTCODES"
Toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwood
1,M4A,North York,Victoria Villag
2,M5A,Downtown Toronto,"Harbourfron,Regent Par"
3,M6A,North York,"Lawrence Height,Lawrence Mano"
4,M7A,Queen's Park,Not assigne
5,M9A,Etobicoke,Islington Avenu
6,M1B,Scarborough,"Roug,Malver"
7,M3B,North York,Don Mills Nort
8,M4B,East York,"Woodbine Garden,Parkview Hil"
9,M5B,Downtown Toronto,"Ryerso,Garden Distric"


In [93]:
# THIS IS 2nd SECTION

LandLfile = 'http://cocl.us/Geospatial_data'
Col_headers = ["Postcode","Latitude","Longitude"]  # 

df_Geo_Toronto = pd.read_csv(LandLfile,names=Col_headers,skiprows=1)  # getting the L& L data
print("Done")

Done


In [94]:
df_Geo_Toronto.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [95]:
df_FINAL=pd.merge(Toronto_df, df_Geo_Toronto, on='Postcode', how='inner')  # merging the dataframes to get the L & L data annexed to the earlier DataFrame

In [97]:
# This is the 3rd section (Clustering & Neighbourthood analysis.....)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

In [ ]:
!conda update conda
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

In [98]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


ModuleNotFoundError: No module named 'geopy'

In [96]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

df_Toronto_Special=df_FINAL[df_FINAL['Borough'].str.contains("Toronto")]   # select Boroughs with "Toronto"
df_Toronto_Special.head()

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


ModuleNotFoundError: No module named 'geopy'

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto_Special['Latitude'], df_Toronto_Special['Longitude'], df_Toronto_Special['Borough'], df_Toronto_Special['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

neighborhoods=df_Toronto_Special
CT_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
CT_data.head()

address = 'Central Toronto, Ontario'  # selecting central Toronto (CT) for deep dive

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_CT = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(CT_data['Latitude'], CT_data['Longitude'], CT_data['Borough'], CT_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CT)  
    
map_CT

CLIENT_ID = 'XDFCVUVM1ICDDJIYMQ2RYMWEV5BT15O00ZUXID1UFCYQ1X2Y' # your Foursquare ID
CLIENT_SECRET = 'WRRZTV5GQ1RYQX3UE5I53EQ0WBSB2KKG5U2G5GGAXJ5UDOF1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CT_data.loc[0, 'Neighbourhood']

neighborhood_latitude = CT_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = CT_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = CT_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

In [6]:
map_toronto

In [7]:
map_CT